In [2]:
! pip install opencv-python==4.5.5.64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60.5 MB 1.1 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.6.0.66
    Uninstalling opencv-python-4.6.0.66:
      Successfully uninstalled opencv-python-4.6.0.66


In [3]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import pickle
import h5py

In [5]:
# script constants
hdf5_dataset_path = r"/content/drive/MyDrive/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/features.hdf5"
sklearn_model_output_path = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap10/saved_model/dogs_cats_logreg.pickle"

In [6]:
db = h5py.File(name=hdf5_dataset_path, mode="r")
i = int(db["labels"].shape[0] * 0.75)
i

18750

In [7]:
print(f"[INFO] tuning hyperparameters......")
params = {'C': [0.0001, 0.001, 0.01, 0.1, 1.0]}
model = GridSearchCV(LogisticRegression(solver="lbfgs", multi_class='auto'), param_grid=params, cv=3, n_jobs=1)
model.fit(db["data"][:i], db["labels"][:i])

print(f"[INFO] best hyperparameters: {model.best_params_}")

[INFO] tuning hyperparameters......


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[INFO] best hyperparameters: {'C': 0.01}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [8]:
print(f"[INFO] best hyperparameters: {model.best_params_}")

[INFO] best hyperparameters: {'C': 0.01}


In [11]:
target_names= [label.decode() for label in db["label_names"]]
target_names

['cat', 'dog']

In [12]:
# generate a classification report for the model
print("[INFO] evaluating...")
preds = model.predict(db["data"][i:])
print(classification_report(db["labels"][i:], preds,
                            target_names=target_names))

# compute the raw accuracy with extra precision
acc = accuracy_score(db["labels"][i:], preds)
print("[INFO] score: {}".format(acc))

[INFO] evaluating...
              precision    recall  f1-score   support

         cat       0.99      0.99      0.99      3110
         dog       0.99      0.99      0.99      3140

    accuracy                           0.99      6250
   macro avg       0.99      0.99      0.99      6250
weighted avg       0.99      0.99      0.99      6250

[INFO] score: 0.99056


In [13]:
print(f"[INFO] saving model......")
f = open(sklearn_model_output_path, mode="wb")
f.write(pickle.dumps(model.best_estimator_))
f.close()
db.close()

[INFO] saving model......
